In [1]:
import pandas as pd
from epiweeks import Week
from datetime import datetime, date
from bulletin.data.notifica import Notifica
from bulletin import __file__ as __root__
from bulletin.metabase.request import download_metabase

In [2]:
notifica = Notifica()

In [3]:
notifica.download_todas_notificacoes()

Requesting classificacao_final IS NULL AND nt.metodo <> '' AND nt.metodo <> '3' AND nt.metodo IS NOT NULL AND nt.excluir_ficha = '2' AND nt.uf_residencia = '41' AND nt.uf_unidade_notifica = '41' AND (nt.status_notificacao = '1' OR nt.status_notificacao = '2')
Success code 200
Saving in input\queries_exames\null.csv
Download finish, time elapsed: 0:00:00.320623
downloaded shape (17, 4)

Requesting classificacao_final = 0 AND nt.metodo <> '' AND nt.metodo <> '3' AND nt.metodo IS NOT NULL AND nt.excluir_ficha = '2' AND nt.uf_residencia = '41' AND nt.uf_unidade_notifica = '41' AND (nt.status_notificacao = '1' OR nt.status_notificacao = '2')
Success code 200
Saving in input\queries_exames\0.csv
Download finish, time elapsed: 0:00:00.070564
downloaded shape (12, 4)

Requesting classificacao_final = 1 AND nt.metodo <> '' AND nt.metodo <> '3' AND nt.metodo IS NOT NULL AND nt.excluir_ficha = '2' AND nt.uf_residencia = '41' AND nt.uf_unidade_notifica = '41' AND (nt.status_notificacao = '1' OR nt

In [4]:
notifica.read_todas_notificacoes()

In [5]:
todos_exames = notifica.get_casos()

In [6]:
#todos_exames.loc[~todos_exames['data_notificacao'].isnull(), 'ano'] = todos_exames['data_notificacao'].loc[~todos_exames['data_notificacao'].isnull()].apply(lambda x: Week.fromdate(x).year)
#todos_exames.loc[~todos_exames['data_notificacao'].isnull(), 'se'] = todos_exames['data_notificacao'].loc[~todos_exames['data_notificacao'].isnull()].apply(lambda x: Week.fromdate(x).week)
todos_exames = todos_exames.loc[~todos_exames['ibge_unidade_notifica'].isnull()]
todos_exames = todos_exames.loc[~todos_exames['data_notificacao'].isnull()]
todos_exames['se'] = todos_exames['data_notificacao'].apply(lambda x: Week.fromdate(x))

In [7]:
#todos_exames
tabela = todos_exames.groupby(['se','classificacao_final'])[['metodo']].count().unstack()

In [8]:
tabela = tabela.droplevel(0, axis=1)

In [9]:
for row in tabela.index.values:
    if row.year < 2020:
        tabela = tabela.drop(row)

In [10]:
tabela['Semana Epidemiológica'] = [ x.week for x in tabela.index.values]
tabela = tabela.set_index('Semana Epidemiológica')

In [11]:
tabela[5] = tabela[5].add(tabela[0])
tabela = tabela.drop(0, axis=1)

In [17]:
tabela = tabela[[2, 3, 1, 5]]
tabela = tabela.rename(columns={2:'Casos confirmados', 3:'Casos descartados', 1:'Casos suspeitos', 5:'Não informados'})

In [19]:
tabela['Total testes'] = tabela.sum(1)
tabela['Positividade (%)'] = tabela['Casos confirmados'].div(tabela['Total testes'])

In [20]:
tabela.to_excel('output/exames_notifica.xlsx', index = False)

In [21]:
tabela

classificacao_final,Casos confirmados,Casos descartados,Casos suspeitos,Não informados,Total testes,Positividade (%)
Semana Epidemiológica,,,,,,
1,4.0,2.0,NaN,NaN,6.0,0.666667
2,2.0,1.0,1.0,NaN,4.0,0.500000
3,3.0,1.0,NaN,NaN,4.0,0.750000
4,3.0,2.0,NaN,NaN,5.0,0.600000
5,4.0,3.0,NaN,NaN,7.0,0.571429
...,...,...,...,...,...,...
13,18370.0,15754.0,13365.0,NaN,47489.0,0.386826
14,19517.0,16834.0,13455.0,NaN,49806.0,0.391860
15,19403.0,18065.0,12773.0,NaN,50241.0,0.386199
